# QA System for the Solow model

# Loading libraries

In [ ]:
!pip install accelerate --q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 6.3 MB/s eta 0:00:00


In [ ]:
%time
!pip install pypdf --q
!pip install -qqq chromadb==0.4.10 --progress-bar off
!pip install -qqq sentence_transformers==2.2.2 --progress-bar off!pip install -Uqqq pip --progress-bar off
# !pip install -qqq torch==2.0.1 --progress-bar off
!pip install -qqq langchain==0.0.299 --progress-bar off
!pip install -qqq xformers==0.0.21 --progress-bar off
!pip install -qqq sentence_transformers==2.2.2 --progress-bar off
!pip install -qqq tokenizers==0.14.0 --progress-bar off
!pip install -qqq optimum==1.13.1 --progress-bar off
!pip install -qqq auto-gptq==0.4.2 --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/ --progress-bar off
!pip install -qqq unstructured==0.10.16 --progress-bar off


CPU times: user 2 µs, sys: 2 µs, total: 4 µs
Wall time: 7.87 µs
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.0/284.0 kB 7.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.

Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project

In [ ]:
from langchain.document_loaders import UnstructuredMarkdownLoader
from langchain.document_loaders import PyPDFLoader
from langchain.llms import HuggingFaceHub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from getpass import getpass
import os
from langchain.chains import RetrievalQA
from langchain import PromptTemplate
from textwrap import fill
import torch
from langchain import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline

# Retrieving data

In [ ]:
loader = PyPDFLoader("https://github.com/AlexanderB111/Deep-Learning/raw/main/Assignment%203/Mankiw%20Solow.pdf")


docs = loader.load()
len(docs)

32

## Splitting the text up in chunk sizes of 1024 tokens.

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=64)
texts = text_splitter.split_documents(docs)
len(texts)

96

## Creating embeddings for the chunks:

In [ ]:
embeddings = HuggingFaceEmbeddings(
    model_name="thenlper/gte-large",
    model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True},
)

query_result = embeddings.embed_query(texts[0].page_content)
print(len(query_result))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/67.9k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

1024


### Store embeddings in database

In [ ]:
db = Chroma.from_documents(texts, embeddings, persist_directory="db")
results = db.similarity_search("Transformer models", k=2)
print(results[0].page_content)

determinants of the ultimate steady state and the initial level of 
 income. 
 Endogenous-growth models make predictions very different 
 from the Solow model regarding convergence among countries. In 
 endogenous-growth models there is no steady-state level of income; 
 differences among countries in income per capita can persist 
 indefinitely, even if the countries have the same saving and 
 population growth rates.12 Endogenous-growth models with a 
 12. Although we do not explore the issue here, endogenous-growth models also 
 make quantitative predictions about the impact of saving on growth. The models are 
 typically characterized by constant returns to reproducible factors of production, 
 namely physical and human capital. Our model of Section II with a + , = 1 and g = 
 0 provides a simple way of analyzing the predictions of models of endogenous


# Build the model

In [ ]:
MODEL_NAME = "TheBloke/Llama-2-7B-Chat-GPTQ"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, torch_dtype=torch.float16, trust_remote_code=True, device_map="auto"
)

# Create a configuration for text generation based on the specified model name
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)

# Set the maximum number of new tokens in the generated text to 1024.
# This limits the length of the generated output to 1024 tokens.
generation_config.max_new_tokens = 1024

# Set the temperature for text generation. Lower values (e.g., 0.0001) make output more deterministic, following likely predictions.
# Higher values make the output more random.
generation_config.temperature = 0.0001

# Set the top-p sampling value. A value of 0.95 means focusing on the most likely words that make up 95% of the probability distribution.
generation_config.top_p = 0.95

# Enable text sampling. When set to True, the model randomly selects words based on their probabilities, introducing randomness.
generation_config.do_sample = True

# Set the repetition penalty. A value of 1.15 discourages the model from repeating the same words or phrases too frequently in the output.
generation_config.repetition_penalty = 1.15


# Create a text generation pipeline using the initialized model, tokenizer, and generation configuration
text_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    generation_config=generation_config,
)

# Create a LangChain pipeline that wraps the text generation pipeline and set a specific temperature for generation
llm = HuggingFacePipeline(pipeline=text_pipeline, model_kwargs={"temperature": 0})


tokenizer_config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

## Design prompt template

In [ ]:
template = """
<s>[INST] <<SYS>>
Act as a Macroeconomic professor. Use the following information to answer the question at the end.
<</SYS>>

{context}

{question} [/INST]
"""

prompt = PromptTemplate(template=template, input_variables=["context", "question"])


qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
)


### Test the model

In [ ]:
result = qa_chain(
    "What is the Solow model? Explain like I am five."
)
print(fill(result["result"].strip(), width=80))

Hey there, little buddy! So, you want to know about the Solow model? Well, it's
actually a really cool economic model that helps us understand how economies
grow and change over time. Think of an economy like a big machine with lots of
different parts working together. The Solow model is like a map that shows how
all these parts fit together and how they work together to make the economy
grow. It helps us understand things like why some countries are richer than
others, or why some people have more money than others. The model is named after
a man named Robert Solow, who created it in the 1950s. He wanted to figure out
why some economies grew faster than others, and he came up with this really
clever way of thinking about it. So, here's the basic idea: imagine you have a
bunch of workers making stuff and trading goods and services with each other.
They all have different skills and abilities, so some can make more stuff than
others. But no matter what, everyone wants to make more money

In [ ]:
result = qa_chain(
    "Summarize the conclusions of the Solow model in 2-3 sentences."
)
print(fill(result["result"].strip(), width=80))

The Solow model predicts that economic growth is driven solely by technological
progress, which increases productivity and leads to an increase in output
without causing inflation. The model also shows that the rate of convergence to
the steady-state value of income is determined by the determinants of the steady
state, such as population growth and technical progress, and that the speed of
convergence can be estimated using the parameter A. Additionally, the model
suggests that the economy will converge to the steady-state value over time, but
this process may take an extended period due to the slow pace of technological
change.


## Try a different prompt

In [ ]:
template = """
<s>[INST] <<SYS>>
Answer the following question using riddles.
<</SYS>>

{context}

{question} [/INST]
"""

prompt = PromptTemplate(template=template, input_variables=["context", "question"])


qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
)


In [ ]:
result = qa_chain(
    "Summarize the conclusions of the Solow model"
)
print(fill(result["result"].strip(), width=80))

Here's a riddle summary of the Solow model's conclusions: In the land of
econometrics, where data reigns supreme, A model was born, its name Solow, a
tale of great dream. It said that savings drive growth, and investment too, But
the rate of return, oh no! That's what it couldn't do. For rich nations see high
returns, while poor ones lag, According to the model, the explanation for this
gap. Now, some say the model's flawed, and endogenous growth they've found, But
the Solow model still stands, its predictions profound. So listen well, young
economist, and heed these words of old, The Solow model's wisdom will guide you,
as you delve into the cold.


# Try a different model

In [ ]:
MODEL_NAME = "openchat/openchat_3.5"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, torch_dtype=torch.float16, trust_remote_code=True, device_map="auto"
)

# Create a configuration for text generation based on the specified model name
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)

# Set the maximum number of new tokens in the generated text to 1024.
# This limits the length of the generated output to 1024 tokens.
generation_config.max_new_tokens = 1024

# Set the temperature for text generation. Lower values (e.g., 0.0001) make output more deterministic, following likely predictions.
# Higher values make the output more random.
generation_config.temperature = 0.0001

# Set the top-p sampling value. A value of 0.95 means focusing on the most likely words that make up 95% of the probability distribution.
generation_config.top_p = 0.95

# Enable text sampling. When set to True, the model randomly selects words based on their probabilities, introducing randomness.
generation_config.do_sample = True

# Set the repetition penalty. A value of 1.15 discourages the model from repeating the same words or phrases too frequently in the output.
generation_config.repetition_penalty = 1.15


# Create a text generation pipeline using the initialized model, tokenizer, and generation configuration
text_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    generation_config=generation_config,
)

# Create a LangChain pipeline that wraps the text generation pipeline and set a specific temperature for generation
llm = HuggingFacePipeline(pipeline=text_pipeline, model_kwargs={"temperature": 0})

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/491 [00:00<?, ?B/s]

...

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/179 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


## Prompt template: Macroeconomic professor

In [ ]:
template = """
<s>[INST] <<SYS>>
Act as a Macroeconomic professor. Use the following information to answer the question at the end.
<</SYS>>

{context}

{question} [/INST]
"""

prompt = PromptTemplate(template=template, input_variables=["context", "question"])


qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
)


In [ ]:
result = qa_chain(
    "What is the Solow model? Explain like I am five."
)
print(fill(result["result"].strip(), width=80))

The Solow model is an economic model that tries to explain how economies grow
over time. It looks at three main factors: savings, population growth, and
technological progress. The model says that if people save more money, the
economy will grow faster. If there are more people in the workforce, the economy
will also grow faster. And if new technologies are developed, the economy will
grow even faster. The Solow model also talks about something called the "steady-
state" which is when the economy reaches its maximum potential growth rate. This
means that no matter what happens, the economy will always grow at this same
rate. So, the Solow model helps us understand why some countries have higher
levels of income than others and why some countries grow faster than others.


In [ ]:
result = qa_chain(
    "Summarize the conclusions of the Solow model in 2-3 sentences."
)
print(fill(result["result"].strip(), width=80))

The Solow growth model suggests that countries with higher savings rates will
grow faster than those with lower savings rates. It also predicts that there
will be conditional convergence, meaning that poorer countries will eventually
catch up to richer ones if they have similar levels of capital accumulation and
technological progress. However, it does not provide specific guidance on how
quickly this convergence will occur.


## Prompt template: Riddle

In [ ]:
template = """
<s>[INST] <<SYS>>
Answer the following question using riddles.
<</SYS>>

{context}

{question} [/INST]
"""

prompt = PromptTemplate(template=template, input_variables=["context", "question"])


qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
)


In [ ]:
result = qa_chain(
    "Summarize the conclusions of the Solow model"
)
print(fill(result["result"].strip(), width=80))

The Solow model suggests that there are three main factors affecting economic
growth: population growth, technological progress, and the accumulation of
physical and human capital. The model also assumes that these factors work
together to produce economic growth, but it does not provide any specific
mechanism through which they interact. Instead, it simply states that if you
increase any one of these factors, then you will get more output.  In addition
to these three main factors, the Solow model also includes some assumptions
about how different types of resources are used in production. For example, it
assumes that labor is always fully employed and that firms always choose the
most efficient way to combine inputs in order to maximize profits. It also
assumes that there is no government intervention in the economy and that markets
function perfectly.  Overall, the Solow model provides a simple framework for
understanding economic growth, but it has its limitations. For instance, it 